In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics


In [2]:
import random

nrow = 10000 #desired sample size
n = sum(1 for line in open('../data-goodmove.csv')) - 1 #number of records in file (excludes header)

skiprows = np.sort(np.random.choice(np.arange(n), n - nrow, replace= False))

In [3]:
df = pd.read_csv('data-goodmove.csv', skiprows=skiprows)
print(df.shape)

(10000, 193)


In [4]:
df = df.replace(['None', 'P', 'N', 'B', 'R', 'Q', 'K', 'p', 'n', 'b', 'r', 'q', 'k'], [0, 1, 3, 3, 5, 9, 100, -1, -3, -3, -5, -9, -100])

In [5]:
X_dataFrame, y_dataFrame = df.iloc[:, :192], df.iloc[:, 192]

In [6]:
from sklearn.preprocessing import OneHotEncoder

X = X_dataFrame.to_numpy()
y = y_dataFrame.to_numpy().reshape(-1, 1)

In [7]:
X_train, X_test , y_train,  y_test = train_test_split(X, y, test_size=0.25)
del (X, y, X_dataFrame, y_dataFrame)

In [8]:
encY = OneHotEncoder().fit(y_train)
encY.inverse_transform([[1, 0], [0, 1]])

array([[0],
       [1]])

In [9]:
Y_train = np.array(encY.transform(y_train).toarray())
Y_train.shape

(7500, 2)

In [10]:
class f1_loss(nn.Module):
  def __init__(self, weight_class = None, weight_tp = None):
    super(f1_loss, self).__init__()
    self.weight_class = weight_class
    self.weight_tp = weight_tp


  def forward(self, Y_pred, Y_true):

    tp = torch.sum(Y_true*Y_pred, dim=0)
    
    fp = torch.sum((1-Y_true)*Y_pred, dim=0)
    fn = torch.sum(Y_true*(1-Y_pred), dim=0)

    if self.weight_tp is not None:
      tp = tp * torch.Tensor(self.weight_tp)

    p = tp / (tp + fp + 1e-10)
    r = tp / (tp + fn + 1e-10)

    f1 = 2*p*r / (p+r+1e-10)
    f1 = torch.where(torch.isnan(f1), torch.zeros_like(f1), f1)
    if self.weight_class is not None:
      # NOTE 
      # f1 = f1 * torch.Tensor(self.weight).cuda()
      f1 = f1 * torch.Tensor(self.weight_class)


    return - torch.mean(f1)

In [11]:
class BuildDataSet(Dataset):
    def __init__ (self, X, Y = np.empty((0, 2))):
        super().__init__()
        self.size = X.shape[0]
        self.data = torch.Tensor(X)
        if len(Y) == 0:
            self.is_testSet = True
        else:
            self.is_testSet = False
            self.label = torch.Tensor(Y)

    
    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        if self.is_testSet:
            return self.data[idx]
        return self.data[idx], self.label[idx]

In [12]:
class DeepLearningModel(nn.Module):
    def __init__(self, in_features):
        super(DeepLearningModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_features = in_features, out_features = 50),
            nn.ReLU(),
            nn.Linear(50, 2),
            # nn.ReLU(),
            nn.Softmax(1)
        )
    
    def forward(self, x):
        out = self.fc(x)
        return out

In [13]:
def Train(num_epochs, model, dataset, loss_func, batch_size = 32):

    loaders = DataLoader(dataset, batch_size)
    total_step = len(loaders)

    model.train()
    optimizer = optim.Adam(model.parameters())
    
    for epoch in range(num_epochs):
        for i, (x, y) in enumerate(loaders):
            # x, y = x.type(torch.float).to('cuda'), y.type(torch.float).to('cuda')
            out = model(x)
            loss = loss_func(out, y)

            optimizer.zero_grad()

            loss.backward()
            optimizer.step()

            if (i+1) % 20 == 0:
                
                Y_pred = Predict(DLmodel, torch.Tensor(X_train))
                y_pred = encY.inverse_transform(Y_pred).reshape(-1,)
                y_label = y_train.reshape(-1, )
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, {}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(), metrics.f1_score(y_label, y_pred, average='macro')))


In [14]:
def Predict(model, X_loaders):
    Y_pred = model(X_loaders).detach().numpy()
    Y_pred = np.where(Y_pred < 0.5, 0, 1)
    return Y_pred

In [15]:
train_dataset =  BuildDataSet(X_train, Y_train)

In [16]:
DLmodel = DeepLearningModel(192)

In [17]:
loss_func = f1_loss([197, 7303], [1, 10])

In [18]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    DLmodel = DLmodel.cuda()
    train_dataset.to('cuda')

Train(400, DLmodel, train_dataset, loss_func, 64)

Epoch [1/400], Step [20/118], Loss: -87.1403, 0.46268678153058274
Epoch [1/400], Step [40/118], Loss: -2865.4167, 0.44238109799044184
Epoch [1/400], Step [60/118], Loss: -268.7968, 0.4279006998518845
Epoch [1/400], Step [80/118], Loss: -1034.9614, 0.428124899094203
Epoch [1/400], Step [100/118], Loss: -1594.0448, 0.410463441080217
Epoch [2/400], Step [20/118], Loss: -68.7886, 0.3906047553665663
Epoch [2/400], Step [40/118], Loss: -2322.6902, 0.24556339027968385
Epoch [2/400], Step [60/118], Loss: -1124.4629, 0.2781090098506226
Epoch [2/400], Step [80/118], Loss: -1937.5781, 0.29816996678936475
Epoch [2/400], Step [100/118], Loss: -1576.1724, 0.4128490105719707
Epoch [3/400], Step [20/118], Loss: -73.5250, 0.4000428419568033
Epoch [3/400], Step [40/118], Loss: -2113.4604, 0.29132895649941404
Epoch [3/400], Step [60/118], Loss: -1121.0452, 0.29532263391383456
Epoch [3/400], Step [80/118], Loss: -1921.0730, 0.29875382830524655
Epoch [3/400], Step [100/118], Loss: -1314.0090, 0.30697938291

In [19]:

Y_pred = Predict(DLmodel, torch.Tensor(X_train))
y_pred = encY.inverse_transform(Y_pred).reshape(-1,)
y_label = y_train.reshape(-1, )

print(confusion_matrix(y_pred, y_label))
print(metrics.f1_score(y_label, y_pred, average='macro'))
print(metrics.classification_report(y_label, y_pred))


[[7216  101]
 [  80  103]]
0.7599567550821531
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7296
           1       0.56      0.50      0.53       204

    accuracy                           0.98      7500
   macro avg       0.77      0.75      0.76      7500
weighted avg       0.97      0.98      0.98      7500



In [20]:
Y_pred = Predict(DLmodel, torch.Tensor(X_test))
y_pred = encY.inverse_transform(Y_pred).reshape(-1,)
y_label = y_test.reshape(-1, )

print(confusion_matrix(y_label, y_pred))
print(metrics.f1_score(y_label, y_pred, average='macro'))
print(metrics.classification_report(y_label, y_pred))

[[2356   71]
 [  70    3]]
0.5058810666240533
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      2427
           1       0.04      0.04      0.04        73

    accuracy                           0.94      2500
   macro avg       0.51      0.51      0.51      2500
weighted avg       0.94      0.94      0.94      2500

